In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pandas as pd
from tqdm import trange
from src.models.error_measures import get_accuracy
from src.models.TwoLayerPerceptron import TwoLayerPerceptron

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [3]:
# setting seeds
np.random.seed(445)
torch.manual_seed(445)

## Data

In [4]:
data06 = pd.read_csv("../data/processed/target06.csv")

In [5]:
all_data = data.TensorDataset(torch.from_numpy((data06.values[:,:-1] - data06.values[:,:-1].min(0)) / data06.values[:,:-1].ptp(0)).float(), torch.from_numpy(data06.values[:,-1]).float())  # with normalization
train_dataset, test_dataset, valid_dataset = torch.utils.data.random_split(all_data, (round(0.7 * len(all_data)), round(0.2 * len(all_data)), round(0.1 * len(all_data))))

In [6]:
data_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
test_data_loader = data.DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=False)
valid_data_loader = data.DataLoader(valid_dataset, batch_size=256, shuffle=False, drop_last=False)

## Prepare model

In [7]:
model = TwoLayerPerceptron(num_inputs=8, num_hidden=500, num_outputs=1)
model.to(device)
print(model)

TwoLayerPerceptron(
  (lin1): Linear(in_features=8, out_features=500, bias=True)
  (bn1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (lin2): Linear(in_features=500, out_features=1, bias=True)
)


In [8]:
loss_fun = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)
epochs = 350

## Training

In [9]:
for epoch in trange(epochs):
    model.train()  # reset in get_accuracy
    for data,targets in data_loader:
        data = data.to(device)
        targets = targets.to(device)
        preds = model(data).squeeze(dim=1)
        loss = loss_fun(preds, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 50 == 0:
        print(f"Epoch: {epoch}, loss: {loss.item():.3}, acc: {get_accuracy(model, valid_data_loader,  device=device, pct_close=0.2)}")

  0%|          | 1/350 [00:05<32:06,  5.52s/it]

Epoch: 0, loss: 5.23e+02, acc: 0.0


 15%|█▍        | 51/350 [00:30<02:18,  2.16it/s]

Epoch: 50, loss: 1.3e+02, acc: 0.2786127167630058


 29%|██▉       | 101/350 [00:54<02:04,  2.00it/s]

...

 72%|███████▏  | 251/350 [02:04<00:42,  2.31it/s]

Epoch: 250, loss: 63.6, acc: 0.34797687861271676


 86%|████████▌ | 301/350 [02:29<00:23,  2.11it/s]

Epoch: 300, loss: 51.7, acc: 0.35722543352601155


100%|██████████| 350/350 [02:54<00:00,  2.01it/s]


In [ ]:
get_accuracy(model, test_data_loader, device=device, pct_close=0.2)

In [ ]:
targets

In [ ]:
preds